In [5]:
import os, sys
from utils import *
from download_DB import download_db

In [17]:
directory = download_db(lista_download=["vRAPEELLicenciamento".lower()])


******************** Baixando arquivos *********************

Carregando arquivo de log...
'vrapeellicenciamento.gzip' salvo em 'C:\Users\viniciusbarros\Documents\Previsor\SKATE_Downloads/2022_12_27/'.

************************************************************



In [7]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]

In [8]:
monitoramento_dados  = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip")).drop_duplicates(inplace=True)
io_rapeel = pd.read_parquet(os.path.join(directory,"vrapeelcronograna.gzip"))[['DthEnvio','IdeUsinaOutorga','DatPrevistaAprovacaoIII','DatRealizacaoIII']].drop_duplicates(inplace=True)

In [9]:
rename_cols = {'DthEnvio' : 'Data_Envio',
'NomUsina' : 'usina_nome',
'DatRealizacaoII' : 'canteiroReal',
'DatPrevistaAprovacaoIII' : 'prev_IO',
'DatRealizacaoIII' : 'IO_real',
'DatRealizacaoIX' : 'DesvRio_real',
'DatRealizacaoXI' : 'Ench_Real',
'DatRealizacaoXIII' : 'IOTrans_Real',
'DatRealizacaoXIV' : 'Conc_Trans_REAL',
'DatRealizacaoXII' : 'Comiss_Real',
'DatRealizacaoVII' : 'ME_Real_conc_eol',
'DatRealizacaoVIII' : 'Conc_Combust_Real',}
cols = ['IdeUsinaOutorga','CodCeg','NomUsina','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV',]
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograna.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [20]:
rename_cols = {
    'DatPrevistaComercial' : 'DatPrevistaComercial1'
}
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','SigTipoGeracao','DscObservacao','VlrPotenciaUnitaria','NumOperacaoUg','DatPrevistaComercial','DatPrevistaRotor','DatPrevistaMaximaRotor','DatRealizacaoRotor','DatPrevistaTeste','DatPrevistaMaximaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatPrevistaMaximaComercial','DatRealizacaoComercial','SigTipoGeracao','NomUsina']
ugrapeel  = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)
ugrapeel = ugrapeel.loc[ugrapeel.groupby('IdeUsinaOutorga')


In [11]:
rename_cols = {
'DthEnvio' : 'Data_Envio',
'SigTipoGeracao' : 'TipoGeracao',
'NomUsina' : 'usina_nome',
'NumOperacaoUg' : 'Num_UG',
'VlrPotenciaUnitaria' : 'Pot_UG',
'DatRealizacaoRotor' : 'UG_decidarotor_real',
'DatPrevistaTeste' : 'UG_op_test_previsão',
'DatRealizacaoTeste' : 'UG_op_test_real',
'DatPrevistaComercial' : 'UG_op_com_previsão',
'DatRealizacaoComercial' : 'UG_op_com_real',
}

cols = ['DthEnvio','IdeUsinaOutorga','SigTipoGeracao','NomUsina','NumOperacaoUg','VlrPotenciaUnitaria','DatRealizacaoRotor','DatPrevistaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatRealizacaoComercial','DscObservacao','DthEnvio','IdeUsinaOutorga','NumOperacaoUg']
inputug = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [12]:
cols = ['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','MdaPotenciaLiberadaTeste','MdaPotenciaLiberadaComercial','NumDespachoTeste','numDespachoComercial','IdcMonitorada','DatDescidaoutorgadaRotor','DatRealizacaoSFGRotor','DatInicioOpTesteOutorgada','DatLiberacaoSFGTeste','DatUGInicioOpComerOutorgado','DatLiberOpComerRealizado','DatPrevisaoSFGComercial']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].drop_duplicates()

In [13]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','NomSituacaoI','PctEnergiaContratoAssinadoI','DatConclusaoI','NomSituacaoII','PctEnergiaContratoAssinadoII','DatConclusaoII','NomSituacaoIII','PctEnergiaContratoAssinadoIII','DatConclusaoIII','DscObservacao']
rename_cols = {'NomSituacaoI' : 'SituacaoACR',
'PctEnergiaContratoAssinadoI' : 'PctEnergiaContratoAssinadoACR',
'DatConclusaoI' : 'DatConclusaoACR',
'NomSituacaoII' : 'SituacaoCCESI',
'PctEnergiaContratoAssinadoII' : 'PctEnergiaContratoAssinadoCCESI',
'DatConclusaoII' : 'DatConclusaoCCESI',
'NomSituacaoIII' : 'SituacaoACL',
'PctEnergiaContratoAssinadoIII' : 'PctEnergiaContratoAssinadoACL',
'DatConclusaoIII' : 'DatConclusaoACL',
'DscObservacao' : 'DscObservacaomercado'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [18]:
cols= ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','DatSolicitacaoI','DatPrevistaEmissaoI','DatEmissaoI','DatValidadeI','NomSituacaoI','DatSolicitacaoIV','DatPrevistaEmissaoIV','DatEmissaoIV','DatValidadeIV','NomSituacaoIV','DatSolicitacaoV','DatPrevistaEmissaoV','DatEmissaoV','DatValidadeV','NomSituacaoV','IdcTipoEntrave','DscSolucao','DscObservacao']
rename_cols = {
'DatSolicitacaoI' : 'DatSolicitacaoLP',
'DatPrevistaEmissaoI' : 'DatPrevistaEmissaoLP',
'DatEmissaoI' : 'DatEmissaoLP',
'DatValidadeI' : 'DatValidadeLP',
'NomSituacaoI' : 'NomSituacaoLP',
'DatSolicitacaoIV' : 'DatSolicitacaoLI',
'DatPrevistaEmissaoIV' : 'DatPrevistaEmissaoLI',
'DatEmissaoIV' : 'DatEmissaoLI',
'DatValidadeIV' : 'DatValidadeLI',
'NomSituacaoIV' : 'NomSituacaoLI',
'DatSolicitacaoV' : 'DatSolicitacaoLO',
'DatPrevistaEmissaoV' : 'DatPrevistaEmissaoLO',
'DatEmissaoV' : 'DatEmissaoLO',
'DatValidadeV' : 'DatValidadeLO',
'NomSituacaoV' : 'NomSituacaoLO',
'DscObservacao' : 'DscObservacaoAmbiental'}
ambientaldatas =  pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [19]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoI','DatAssinaturaContratoI','DatValidadeContratoI','NomSitContratoII','DatAssinaturaContratoII','DatValidadeContratoII','NomSituacaoContratoIII','DatAssinaturaContratoIII','DatValidadeContratoIII','NomSitContratoIV','DatAssinaturaContratoIV','DatValidadeContratoIV','DscObservacao']
rename_cols = {'NomSitContratoI' : 'NomSitContratoCCD',
'DatAssinaturaContratoI' : 'DatAssinaturaContratoCCD',
'DatValidadeContratoI' : 'DatValidadeContratoCCD',
'NomSitContratoII' : 'NomSitContratoCCT',
'DatAssinaturaContratoII' : 'DatAssinaturaContratoCCT',
'DatValidadeContratoII' : 'DatValidadeContratoCCT',
'NomSituacaoContratoIII' : 'NomSituacaoContratoCUSD',
'DatAssinaturaContratoIII' : 'DatAssinaturaContratoCUSD',
'DatValidadeContratoIII' : 'DatValidadeContratoCUSD',
'NomSitContratoIV' : 'NomSitContratoCUST',
'DatAssinaturaContratoIV' : 'DatAssinaturaContratoCUST',
'DatValidadeContratoIV' : 'DatValidadeContratoCUST',
'DscObservacao' : 'DscObservacaoAcesso'}
acessocontratos =  pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [46]:
cols1 = ['IdeUsinaOutorga','DthEnvio','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','DscObservacaomercado']
cols2 = ['IdeUsinaOutorga','DthEnvio','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAmbiental']
inputrapeelv = pd.merge(mercado[cols1],ambientaldatas[cols2],on=["IdeUsinaOutorga","DthEnvio"])


In [50]:
inputrapeelv[inputrapeelv.duplicated(subset=["IdeUsinaOutorga","DthEnvio"],keep=False)]

,IdeUsinaOutorga,DthEnvio,CodCeg,NomUsina,PctAoACR,PctAoACL,PctAutoproducao,SituacaoACR,PctEnergiaContratoAssinadoACR,DatConclusaoACR,...,DatValidadeLI,NomSituacaoLI,DatSolicitacaoLO,DatPrevistaEmissaoLO,DatEmissaoLO,DatValidadeLO,NomSituacaoLO,IdcTipoEntrave,DscSolucao,DscObservacaoAmbiental
25569,43196,2022-05-05 20:00:40.663,UTEAIBA043196-6,Sérgio Paranhos,0.0,60.0,40.0,Não vendeu energia nesse ambiente,NaN,NaT,...,2023-02-25,Válida,NaT,2022-08-31,NaT,NaT,Não Solicitada,Não há pendências,None,None
25570,43196,2022-05-05 20:00:40.663,UTEAIBA043196-6,Sérgio Paranhos,0.0,60.0,40.0,Não vendeu energia nesse ambiente,NaN,NaT,...,2023-02-25,Válida,NaT,NaT,NaT,NaT,Não Solicitada,Não há pendências,None,None


In [39]:
mercado[mercado.IdeUsinaOutorga.duplicated()].sort_values(by = "IdeUsinaOutorga")

,DthEnvio,IdeUsinaOutorga,CodCeg,NomUsina,PctAoACR,PctAoACL,PctAutoproducao,SituacaoACR,PctEnergiaContratoAssinadoACR,DatConclusaoACR,SituacaoCCESI,PctEnergiaContratoAssinadoCCESI,DatConclusaoCCESI,SituacaoACL,PctEnergiaContratoAssinadoACL,DatConclusaoACL,DscObservacaomercado
11381,2021-05-05 22:13:20.857,273,PCHPHPR000273-9,Boa Vista II,45.0,55.0,0.0,Todos assinados,100.0,2019-02-26,None,NaN,NaT,Todos assinados,100.0,2017-01-02,None
30750,2022-08-05 10:46:45.623,273,PCHPHPR000273-9,Boa Vista II,45.0,55.0,0.0,Todos assinados,100.0,2019-02-26,None,NaN,NaT,Todos assinados,100.0,2017-01-02,None
9553,2021-03-05 19:48:51.573,273,PCHPHPR000273-9,Boa Vista II,45.0,55.0,0.0,Todos assinados,100.0,2019-02-26,None,NaN,NaT,Todos assinados,100.0,2017-01-02,None
25663,2022-05-05 22:20:50.997,273,PCHPHPR000273-9,Boa Vista II,45.0,55.0,0.0,Todos assinados,100.0,2019-02-26,None,NaN,NaT,Todos assinados,100.0,2017-01-02,None
2981,2020-07-05 17:57:02.390,273,PCHPHPR000273-9,Boa Vista II,45.0,55.0,0.0,Todos assinados,100.0,2019-02-26,None,NaN,NaT,Todos assinados,100.0,2017-01-02,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38346,2022-12-04 07:59:43.697,57329,EOLCVPB057329-9,Frei Damião IV,0.0,100.0,0.0,Não vendeu energia nesse ambiente,NaN,NaT,None,NaN,NaT,Não estão todos assinados,0.0,NaT,None
38348,2022-12-04 08:05:49.030,57330,EOLCVPB057330-2,Frei Damião V,0.0,100.0,0.0,Não vendeu energia nesse ambiente,NaN,NaT,None,NaN,NaT,Não estão todos assinados,0.0,NaT,None
34425,2022-10-05 10:12:10.563,62917,EOLCVBA062917-0,Ventos de Santa Esperança 04,0.0,100.0,0.0,Não vendeu energia nesse ambiente,NaN,NaT,None,NaN,NaT,Não estão todos assinados,NaN,NaT,None
36677,2022-11-04 17:22:49.867,62917,EOLCVBA062917-0,Ventos de Santa Esperança 04,0.0,100.0,0.0,Não vendeu energia nesse ambiente,NaN,NaT,None,NaN,NaT,Não estão todos assinados,NaN,NaT,None
